Connect to database

In [11]:
from query_database import QueryDatabase
from database import Database
from constants import DATABASE_PATH

df = QueryDatabase("SELECT * FROM supplier_invoice.invoice_202408;").df
df.head()

CatalogException: Catalog Error: Table with name invoice_202408 does not exist!
Did you mean "goteborg_stad.pg_catalog.pg_views"?
LINE 1: SELECT * FROM supplier_invoice.invoice_202408;
                      ^

In [5]:
df.shape

NameError: name 'df' is not defined

# Clean cost column

In [ ]:
df.loc[:, "Belopp exkl moms"] = df["Belopp exkl moms"].str[:-3].str.replace(" ", "").astype(int)
df.head()

In [ ]:
with Database(DATABASE_PATH) as db:
    db.connection.register('df', df)
    
    db.query("CREATE SCHEMA IF NOT EXISTS refined_layer;")
    db.query(
        f"""                
    CREATE TABLE IF NOT EXISTS refined_layer.invoice_202408
    AS SELECT * FROM df;
    """
    )

In [6]:
QueryDatabase("SELECT * FROM refined_layer.invoice_202408;").df

CatalogException: Catalog Error: Table with name invoice_202408 does not exist!
Did you mean "pg_catalog.pg_views"?
LINE 1: SELECT * FROM refined_layer.invoice_202408;
                      ^

In [ ]:
QueryDatabase(
    """
    SELECT 
        leverantör, 
        SUM("Belopp exkl moms") AS "Summa (kr)"
        FROM 
            refined_layer.invoice_202408 
        GROUP BY 
            leverantör
        ORDER BY 
            "Summa (kr)"
        DESC
        ;
    """
).df